#### Key setup

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()
api_key = os.getenv("GROQ_API_KEY")

#### Imports

In [3]:
from langchain_groq import ChatGroq
import requests
from langchain_core.tools import tool
from langchain_community.tools import DuckDuckGoSearchRun

#### Search Tool

In [5]:
search_tool = DuckDuckGoSearchRun()
result = search_tool.invoke("top news in india today?")
print(result)

India News | Latest India News | Read latest and breaking news from India. Today's top India news headlines, news on Indian politics, elections, government, business, technology, and Bollywood. Top News Stories of the day, Latest News Headlines, News Specials, Breaking News and Latest India News, World current affairs & Political News all around the clock at NDTV.com. Get the latest India news, breaking news of today, and top updates of Bharat. Stay informed with real-time coverage and analysis of current news in India at news18. Get updated with current top news stories from India and the world only on Zee News. Zee News brings latest news from India and World on breaking news, today news headlines, politics, business, technology, bollywood, entertainment, sports and others. Find exclusive news stories on Indian politics, current affairs, cricket matches, festivals and ... Headlines today: Get news headlines from India and around the world. Today's breaking news on politics, sports, e

#### LLM 

In [6]:

model = ChatGroq(model="llama-3.3-70b-versatile", api_key=api_key)
model.invoke("hello")

AIMessage(content='Hello. How can I assist you today?', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 36, 'total_tokens': 46, 'completion_time': 0.036363636, 'prompt_time': 0.001714647, 'queue_time': 0.055406203, 'total_time': 0.038078283}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_2ddfbb0da0', 'finish_reason': 'stop', 'logprobs': None}, id='run--70e181f8-07c5-42cd-a8ad-5b8792efb238-0', usage_metadata={'input_tokens': 36, 'output_tokens': 10, 'total_tokens': 46})

#### Agent imports

In [7]:
from langchain.agents import create_react_agent, AgentExecutor
from langchain import hub # hub has pre made prompts 

#### Pulling Pre-made Prompt from HUB

In [8]:
#pull ReAct prompt from langchain hub
prompt = hub.pull("hwchase17/react") # pull the standard ReAct agent prompt ( reasoning + action)

#### Create ReAct agent manually with the pulled prompt

In [ ]:
agent = create_react_agent(
    llm = model, 
    tools=[search_tool], # which tools agent has access to
    prompt=prompt
)

###### Agent does the planning and reasoning and agent executer does the work on ground
#### wrap agent with Agent Executer

In [ ]:
agent_executer = AgentExecutor(
    agent=agent,
    tools=[search_tool],
    verbose=True # we will see what agent thinks 
)

#### Invoke the Agent Executor

In [12]:
response = agent_executer.invoke({"input":"3 ways to reach Varanasi from Delhi"})
print(response)



> Entering new AgentExecutor chain...
Thought: To answer this question, I need to find out the different modes of transportation available from Delhi to Varanasi. 

Action: duckduckgo_search
Action Input: Delhi to Varanasi transportationDelhi to Varanasi bus booking starts @ ₹597 - Get best ticket price on sleeper, volvo, private, government bus ticket booking from Delhi to Varanasi. Use Coupon Code IDBIRIDE and get discount upto ₹500 and earn reward points on Delhi to Varanasi bus ticket booking with Adani OneApp. Delhi to Varanasi trip time depends on the dynamic aspects of traffic, road conditions, and change in climate. A network of operators regularly operates buses on this route, significantly contributing to the travel convenience of many people across India. ... This vast fleet of buses assures efficient transportation and accommodates a wide ... Top bus operators for Delhi to Varanasi bus route are: R.S Yadav Travels, Royal Travels Delhi, Safar Express Tour And Travels, Expr

In [14]:
print(response["output"])

There are three ways to reach Varanasi from Delhi: 
1. By Bus: The bus journey from Delhi to Varanasi takes around 13-14 hours and costs between ₹597 - ₹5000, depending on the operator and facilities. Several bus operators, such as R.S Yadav Travels, Royal Travels Delhi, and UPSRTC, operate on this route.
2. By Flight: The flight journey from Delhi to Varanasi takes around 2 hours and 26 minutes. This is the fastest way to reach Varanasi from Delhi.
3. By Train: The train journey from Delhi to Varanasi takes around the same time as the bus journey, but it is a more inexpensive option, with prices starting from ₹415 for a one-way journey. The Swatantrta S Exp (12562_1) is one of the trains that operate on this route.
